In [66]:
import os, glob, time

In [9]:
def get_file_last_time_of_change(fname):
    try:
        mtime = os.path.getmtime(fname)
    except OSError:
        mtime = 0
    return mtime

In [83]:
def process_changed_files(list_of_changed_files):
    for fname in list_of_changed_files:
        print(fname)

In [ ]:
def update_trade():
    

In [ ]:
def update_current():
    

In [88]:
os.chdir(os.getcwd())
last_changed_times = []
last_changed_times_tmp = []
list_of_changed_files = []

for fname in files_name:
        last_changed_times_tmp.append(get_file_last_time_of_change(fname))
files_name = glob.glob(".\data\*.dat")

for i in range(120):     #while True:
    for fname in files_name:
        last_changed_times.append(get_file_last_time_of_change(fname))
    for j in range(len(files_name)):
        if last_changed_times_tmp[j] != last_changed_times[j]:
            list_of_changed_files.append(files_name[j])
            #print(files_name[j], last_changed_times[j], last_changed_times_tmp[j])
    last_changed_times_tmp = last_changed_times
    last_changed_times = []
    
    process_changed_files(list_of_changed_files)
    list_of_changed_files = []
    
    time.sleep(1)
    

.\data\0~Poloniex~BTC~USD.dat
.\data\2~Poloniex~BTC~USD.dat
.\data\0~Poloniex~ETH~USD.dat
.\data\2~Poloniex~ETH~USD.dat
.\data\0~Poloniex~LTC~BTC.dat
.\data\2~Poloniex~LTC~BTC.dat
.\data\0~Poloniex~BTC~USD.dat
.\data\2~Poloniex~BTC~USD.dat
.\data\0~Poloniex~ETH~BTC.dat
.\data\2~Poloniex~ETH~BTC.dat
.\data\0~Poloniex~ETH~USD.dat
.\data\2~Poloniex~ETH~USD.dat
.\data\0~Poloniex~ETH~USD.dat
.\data\0~Poloniex~LTC~BTC.dat
.\data\2~Poloniex~ETH~USD.dat
.\data\2~Poloniex~LTC~BTC.dat
.\data\0~Poloniex~ETH~BTC.dat
.\data\2~Poloniex~ETH~BTC.dat
.\data\0~Poloniex~BTC~USD.dat
.\data\2~Poloniex~BTC~USD.dat


In [74]:
last_changed_times_tmp

[1498224158.3308072,
 1498224136.2441182,
 1498224124.0571418,
 1498224174.183735,
 1498224116.0741751,
 1498224158.3308072,
 1498224136.2441182,
 1498224124.0586421,
 1498224174.1932359,
 1498224116.0766752]

In [75]:
last_changed_times

[]